In [1]:
%matplotlib inline

In [2]:
import pathlib

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from matplotlib import pyplot as plt

In [6]:
import numpy as np
import xarray as xr
from cartopy import crs as ccrs

In [7]:
def regrid(ds_in, target_ds,  method='bilinear'):
    import xesmf as xe
    """Convenience function for one-time regridding"""
    regridder = xe.Regridder(ds_in, target_ds, method, periodic=True)
    ds_out = regridder(ds_in)
    regridder.clean_weight_file()
    return ds_out

In [8]:
def make_forecast_valid_time(dset, nsteps): 
    from dateutil.relativedelta import relativedelta
    import pandas as pd
    import numpy as np
    time = dset.time.to_series()
    tv = [x + relativedelta(months=m + 2) for m in range(1, nsteps + 1) for x in time]
    tv = np.array(tv)
    tv = tv.reshape((len(tv) // nsteps,nsteps))
    dset['forecast_valid_time'] = (('time','step'), tv)
    return dset

In [9]:
xr.set_options(display_style="html")

In [10]:
HOME = pathlib.Path.home()

In [11]:
HOME

PosixPath('/home/nicolasf')

### read the grid for interpolation 

In [12]:
out_grid = xr.open_dataset(HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'GCMs' / 'target_grid' / 'target_grid_2.5x2.5.nc') 

In [13]:
out_grid

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144)
Coordinates:
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Data variables:
    *empty*

### parameters definition here 

In [14]:
# Parameters 
stage_in = 'pre_processed'
stage_out = 'processed'
period = 'hindcasts'
provider = 'CDS'
GCM = 'ECMWF'
var_name = 'T2M'

In [15]:
# Parameters
stage_in = "pre_processed"
stage_out = "processed"
period = "hindcasts"
provider = "CDS"
GCM = "ECMWF"
var_name = "PRECIP"


----- 

In [16]:
ipath = HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'GCMs' / stage_in / period / provider / GCM / var_name

In [17]:
opath = HOME / 'drives' / 'auck_projects' / 'END19101' / 'Working' / 'data' / 'GCMs' / stage_out / period / provider / GCM / var_name

In [18]:
clim_path = opath / 'CLIMATOLOGY' 

In [19]:
if not clim_path.exists(): 
    clim_path.mkdir(parents=True)

In [20]:
lfiles = list(ipath.glob("*.nc"))

In [21]:
lfiles.sort()

In [22]:
lfiles[0]

PosixPath('/home/nicolasf/drives/auck_projects/END19101/Working/data/GCMs/pre_processed/hindcasts/CDS/ECMWF/PRECIP/ECMWF_PRECIP_1993_01.nc')

In [23]:
lfiles[-1]

PosixPath('/home/nicolasf/drives/auck_projects/END19101/Working/data/GCMs/pre_processed/hindcasts/CDS/ECMWF/PRECIP/ECMWF_PRECIP_2016_12.nc')

In [24]:
len(lfiles)

288

In [25]:
dset = xr.open_dataset(lfiles[0])

In [26]:
dset

<xarray.Dataset>
Dimensions:     (lat: 181, lon: 360, member: 25, step: 5)
Coordinates:
    surface     int64 ...
  * step        (step) int64 1 2 3 4 5
  * member      (member) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23 24
  * lat         (lat) float64 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon         (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    time        datetime64[ns] ...
    valid_time  (step) datetime64[ns] ...
Data variables:
    precip      (member, step, lat, lon) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2020-02-03T14:10:11 GRIB to CDM+CF via cfgrib-0....

In [27]:
dset_m = xr.open_mfdataset(lfiles, combine='nested', concat_dim='time')

In [28]:
dset_m

,Array,Chunk
Bytes,11.52 kB,40 B
Shape,"(288, 5)","(1, 5)"
Count,1148 Tasks,288 Chunks
Type,datetime64[ns],numpy.ndarray
,Array,Chunk
Bytes,9.38 GB,32.58 MB
Shape,"(288, 25, 5, 181, 360)","(1, 25, 5, 181, 360)"
Count,1152 Tasks,288 Chunks
Type,float32,numpy.ndarray


In [29]:
dset_m.nbytes / 1e9

9.3830584

In [30]:
if GCM == 'UKMO': 
    dset_m = dset_m.sel(lat=slice(-89.5, None))

### calculate the monthly climatology 

In [31]:
clim_m = dset_m.sel(time=slice('1993','2016'))

### can calculate the lead-time dependent climatology from the ensemble mean 

In [32]:
clim_m = clim_m.mean('member')

In [33]:
clim_m = clim_m.groupby(clim_m.time.dt.month).mean() 

In [34]:
clim_m

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, month: 12, step: 5)
Coordinates:
    surface  int64 0
  * step     (step) int64 1 2 3 4 5
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip   (month, step, lat, lon) float32 dask.array<chunksize=(1, 5, 181, 360), meta=np.ndarray>

### saves to disk in native grid 

In [35]:
clim_m.to_netcdf(clim_path / f'monthly_climatology_1993_2016_{GCM}_{var_name}.nc') 

### now interpolate 

In [36]:
clim_m_regrid = regrid(clim_m, out_grid)

Create weight file: bilinear_181x360_73x144_peri.nc


using dimensions ('lat', 'lon') from data variable precip as the horizontal dimensions for this dataset.
Remove file bilinear_181x360_73x144_peri.nc


In [37]:
clim_m_regrid

<xarray.Dataset>
Dimensions:  (lat: 73, lon: 144, month: 12, step: 5)
Coordinates:
    surface  int64 0
  * step     (step) int64 1 2 3 4 5
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * lon      (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat      (lat) float64 -90.0 -87.5 -85.0 -82.5 -80.0 ... 82.5 85.0 87.5 90.0
Data variables:
    precip   (month, step, lat, lon) float64 dask.array<chunksize=(1, 5, 73, 144), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [38]:
clim_m.to_netcdf(clim_path / f'monthly_climatology_1993_2016_{GCM}_{var_name}_interp.nc') 

### now calculates the seasonal averages 

In [39]:
dset_s = dset_m.rolling(step=3, min_periods=3).mean()

In [40]:
dset_s = dset_s.sel(step=slice(3, None))

In [41]:
dset_s.nbytes / 1e9

5.629837776

In [42]:
dset_s.valid_time.load()

<xarray.DataArray 'valid_time' (time: 288, step: 3)>
array([['1993-05-01T00:00:00.000000000', '1993-06-01T00:00:00.000000000',
        '1993-07-01T00:00:00.000000000'],
       ['1993-06-01T00:00:00.000000000', '1993-07-01T00:00:00.000000000',
        '1993-08-01T00:00:00.000000000'],
       ['1993-07-01T00:00:00.000000000', '1993-08-01T00:00:00.000000000',
        '1993-09-01T00:00:00.000000000'],
       ['1993-08-01T00:00:00.000000000', '1993-09-01T00:00:00.000000000',
        '1993-10-01T00:00:00.000000000'],
       ['1993-09-01T00:00:00.000000000', '1993-10-01T00:00:00.000000000',
        '1993-11-01T00:00:00.000000000'],
       ['1993-10-01T00:00:00.000000000', '1993-11-01T00:00:00.000000000',
        '1993-12-01T00:00:00.000000000'],
       ['1993-11-01T00:00:00.000000000', '1993-12-01T00:00:00.000000000',
        '1994-01-01T00:00:00.000000000'],
       ['1993-12-01T00:00:00.000000000', '1994-01-01T00:00:00.000000000',
        '1994-02-01T00:00:00.000000000'],
       ['1994-01-01T00:00:00.000000000', '1994-02-01T00:00:00.000000000',
        '1994-03-01T00:00:00.000000000'],
       ['1994-02-01T00:00:00.000000000', '1994-03-01T00:00:00.000000000',
        '1994-04-01T00:00:00.000000000'],
       ['1994-03-01T00:00:00.000000000', '1994-04-01T00:00:00.000000000',
        '1994-05-01T00:00:00.000000000'],
       ['1994-04-01T00:00:00.000000000', '1994-05-01T00:00:00.000000000',
        '1994-06-01T00:00:00.000000000'],
       ['1994-05-01T00:00:00.000000000', '1994-06-01T00:00:00.000000000',
        '1994-07-01T00:00:00.000000000'],
       ['1994-06-01T00:00:00.000000000', '1994-07-01T00:00:00.000000000',
        '1994-08-01T00:00:00.000000000'],
       ['1994-07-01T00:00:00.000000000', '1994-08-01T00:00:00.000000000',
        '1994-09-01T00:00:00.000000000'],
       ['1994-08-01T00:00:00.000000000', '1994-09-01T00:00:00.000000000',
        '1994-10-01T00:00:00.000000000'],
       ['1994-09-01T00:00:00.000000000', '1994-10-01T00:00:00.000000000',
        '1994-11-01T00:00:00.000000000'],
       ['1994-10-01T00:00:00.000000000', '1994-11-01T00:00:00.000000000',
        '1994-12-01T00:00:00.000000000'],
       ['1994-11-01T00:00:00.000000000', '1994-12-01T00:00:00.000000000',
        '1995-01-01T00:00:00.000000000'],
       ['1994-12-01T00:00:00.000000000', '1995-01-01T00:00:00.000000000',
        '1995-02-01T00:00:00.000000000'],
       ['1995-01-01T00:00:00.000000000', '1995-02-01T00:00:00.000000000',
        '1995-03-01T00:00:00.000000000'],
       ['1995-02-01T00:00:00.000000000', '1995-03-01T00:00:00.000000000',
        '1995-04-01T00:00:00.000000000'],
       ['1995-03-01T00:00:00.000000000', '1995-04-01T00:00:00.000000000',
        '1995-05-01T00:00:00.000000000'],
       ['1995-04-01T00:00:00.000000000', '1995-05-01T00:00:00.000000000',
        '1995-06-01T00:00:00.000000000'],
       ['1995-05-01T00:00:00.000000000', '1995-06-01T00:00:00.000000000',
        '1995-07-01T00:00:00.000000000'],
       ['1995-06-01T00:00:00.000000000', '1995-07-01T00:00:00.000000000',
        '1995-08-01T00:00:00.000000000'],
       ['1995-07-01T00:00:00.000000000', '1995-08-01T00:00:00.000000000',
        '1995-09-01T00:00:00.000000000'],
       ['1995-08-01T00:00:00.000000000', '1995-09-01T00:00:00.000000000',
        '1995-10-01T00:00:00.000000000'],
       ['1995-09-01T00:00:00.000000000', '1995-10-01T00:00:00.000000000',
        '1995-11-01T00:00:00.000000000'],
       ['1995-10-01T00:00:00.000000000', '1995-11-01T00:00:00.000000000',
        '1995-12-01T00:00:00.000000000'],
       ['1995-11-01T00:00:00.000000000', '1995-12-01T00:00:00.000000000',
        '1996-01-01T00:00:00.000000000'],
       ['1995-12-01T00:00:00.000000000', '1996-01-01T00:00:00.000000000',
        '1996-02-01T00:00:00.000000000'],
       ['1996-01-01T00:00:00.000000000', '1996-02-01T00:00:00.000000000',
        '1996-03-01T00:00:00.000000000'],
       ['1996-02-01T00:00:00.000000000', '1996-03-01T00:00:00.000000000',
        '1996-04-01T00:00:00.000000000'],
   

### now calculate the seasonal climatology 

In [43]:
clim_s = dset_s.sel(time=slice('1993','2016')).mean('member')

In [44]:
clim_s = clim_s.groupby(clim_s.time.dt.month).mean()

In [45]:
clim_s

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, month: 12, step: 3)
Coordinates:
    surface  int64 0
  * step     (step) int64 3 4 5
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip   (month, step, lat, lon) float32 dask.array<chunksize=(1, 3, 181, 360), meta=np.ndarray>

In [46]:
clim_s

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360, month: 12, step: 3)
Coordinates:
    surface  int64 0
  * step     (step) int64 3 4 5
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lat      (lat) float64 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    precip   (month, step, lat, lon) float32 dask.array<chunksize=(1, 3, 181, 360), meta=np.ndarray>

### saves to disk 

In [47]:
clim_s.to_netcdf(clim_path / f'seasonal_climatology_1993_2016_{GCM}_{var_name}.nc') 

### interpolate 

In [48]:
clim_s_regrid = regrid(clim_s, out_grid)

Create weight file: bilinear_181x360_73x144_peri.nc


using dimensions ('lat', 'lon') from data variable precip as the horizontal dimensions for this dataset.
Remove file bilinear_181x360_73x144_peri.nc


In [49]:
clim_s_regrid.to_netcdf(clim_path / f'seasonal_climatology_1993_2016_{GCM}_{var_name}_interp.nc') 

### now calculates the anomalies 

### monthly anomalies 

In [50]:
anoms_monthly = dset_m.groupby(dset_m.time.dt.month) - clim_m

In [51]:
anoms_monthly

,Array,Chunk
Bytes,11.52 kB,40 B
Shape,"(288, 5)","(1, 5)"
Count,2012 Tasks,288 Chunks
Type,datetime64[ns],numpy.ndarray
,Array,Chunk
Bytes,9.38 GB,32.58 MB
Shape,"(288, 25, 5, 181, 360)","(1, 25, 5, 181, 360)"
Count,3612 Tasks,288 Chunks
Type,float32,numpy.ndarray


### seasonal anomalies 

In [52]:
anoms_seasonal = dset_s.groupby(dset_s.time.dt.month) - clim_s

### regrid 

In [53]:
anoms_monthly_regrid = regrid(anoms_monthly[[var_name.lower()]], out_grid)

Create weight file: bilinear_181x360_73x144_peri.nc


using dimensions ('lat', 'lon') from data variable precip as the horizontal dimensions for this dataset.
Remove file bilinear_181x360_73x144_peri.nc


In [54]:
anoms_seasonal_regrid = regrid(anoms_seasonal[[var_name.lower()]], out_grid)

Create weight file: bilinear_181x360_73x144_peri.nc


using dimensions ('lat', 'lon') from data variable precip as the horizontal dimensions for this dataset.
Remove file bilinear_181x360_73x144_peri.nc


In [55]:
anoms_monthly

,Array,Chunk
Bytes,11.52 kB,40 B
Shape,"(288, 5)","(1, 5)"
Count,2012 Tasks,288 Chunks
Type,datetime64[ns],numpy.ndarray
,Array,Chunk
Bytes,9.38 GB,32.58 MB
Shape,"(288, 25, 5, 181, 360)","(1, 25, 5, 181, 360)"
Count,3612 Tasks,288 Chunks
Type,float32,numpy.ndarray


### add the valid time to the anomalies datasets

In [56]:
anoms_monthly_regrid['valid_time'] = anoms_monthly['valid_time']

In [57]:
anoms_seasonal_regrid['valid_time'] = anoms_seasonal['valid_time']

### now saves to disk 

In [58]:
opath

PosixPath('/home/nicolasf/drives/auck_projects/END19101/Working/data/GCMs/processed/hindcasts/CDS/ECMWF/PRECIP')

In [59]:
for date in pd.to_datetime(anoms_monthly.time.data): 
    print(f"processing monthly dataset for date {date:%Y_%m}")
    sub = anoms_monthly.sel(time=date)
    sub.to_netcdf(opath / f"{GCM}_{var_name}_monthly_anomalies_{date:%Y_%m}.nc")
    sub.close() 
    sub_regrid = anoms_monthly_regrid.sel(time=date)
    sub_regrid.to_netcdf(opath / f"{GCM}_{var_name}_monthly_anomalies_interp_{date:%Y_%m}.nc")
    sub_regrid.close()

processing monthly dataset for date 1993_01


processing monthly dataset for date 1993_02


processing monthly dataset for date 1993_03


processing monthly dataset for date 1993_04


processing monthly dataset for date 1993_05


...

processing monthly dataset for date 2012_01


processing monthly dataset for date 2012_02


processing monthly dataset for date 2012_03


processing monthly dataset for date 2012_04


processing monthly dataset for date 2012_05


In [ ]:
for date in pd.to_datetime(anoms_seasonal.time.data): 
    print(f"processing seasonal dataset for date {date:%Y_%m}")
    sub = anoms_seasonal.sel(time=date)
    sub.to_netcdf(opath / f"{GCM}_{var_name}_seasonal_anomalies_{date:%Y_%m}.nc")
    sub.close() 
    sub_regrid = anoms_seasonal_regrid.sel(time=date)
    sub_regrid.to_netcdf(opath / f"{GCM}_{var_name}_seasonal_anomalies_interp_{date:%Y_%m}.nc")
    sub_regrid.close()